In [ ]:
import rioxarray
import xarray as xr
import glob
from shapely.geometry import box
from datetime import datetime
import numpy as np

## Merge GEDI rh98 with GLAD ARD
1. Open GLAD ARD annual mosaics built in `GLAD_ARD.ipynb` from 16 day-interval scenes of pre-normalized/harmonized landsat data from landsat4->landsat8 giving us temporal coverage of 1997-2023.
2. Open GEDI rh98 tifs created by earth engine and dumped to drive here: https://code.earthengine.google.com/f60eee0697f93ad993a2ac91c505dfdf
3. Reproject/Resample GLAD ARD to match GEDI (We try to avoid resampling/reprojecting the GEDI data if possible since geolocation/resampling errors may effect our estimates of model performance)

Gedi tifs and `105W_39N_annual_median_composite.zarr` are uploaded to our team data-store [here](https://de.cyverse.org/data/ds/iplant/home/shared/earthlab/forest_carbon_codefest/Team_outputs/Team2?selectedOrder=asc&selectedOrderBy=name&selectedPage=0&selectedRowsPerPage=100).

In [ ]:
# copy from team data store
ard = xr.open_zarr("105W_39N_annual_median_composite.zarr")
aoi = box(*ard.rio.bounds())

In [ ]:
# copy from team data store
gedi_paths = glob.glob("../glad_ard/gedi/*")

In [ ]:
# build temporal mosaics simply by adding time coord and stacking along time axis
gedi_dsets = []
for f in gedi_paths:
    dset = rioxarray.open_rasterio(f, chunks={"x": 1024,"y":1024})
    dset = dset.expand_dims("time")
    year = int(f.split("_")[-1].split(".")[0])
    gedi_dsets.append(dset.assign_coords(time=("time", [datetime(year=year, month=12, day=31)])))

gedi = xr.concat(gedi_dsets, dim="time")
gedi = gedi.assign_coords(band=('band', ['rh98'])).to_dataset("band")

In [ ]:
from tqdm.auto import tqdm

In [ ]:
# reproject ard to match gedi
reprojected_dsets = []
for year in tqdm(ard.time.data):
    ard_reprojected = ard.sel(time=year).rio.reproject_match(gedi)
    rr = ard_reprojected.expand_dims("time").assign_coords(time=("time", [year]))
    reprojected_dsets.append(rr)
reprojected_ard = xr.concat(reprojected_dsets, dim="time")
ard_reprojected = reprojected_ard['__xarray_dataarray_variable__'].to_dataset("band")

In [ ]:
# let's add variable names for ard and merge ard and gedi together since they now have
# matching bounds + geo transform
array = ard_reprojected.to_array()
ard_reprojected = array.assign_coords(variable=('variable', ["Blue",'Green','Red','NIR','SWIR1','SWIR2','Thermal','QF'])).to_dataset('variable')
combined = xr.merge([ard_reprojected.chunk({'x': 1024, 'y': 1024, 'time': 1}), gedi.chunk({'x': 1024, 'y': 1024, 'time': 1})])

In [ ]:
# write out the dataset to avoid having to reproject/resample/rechunk/merge again.
combined.chunk({'x': 1024, 'y': 1024, 'time': 1}).to_zarr("../glad_ard/ard_gedi.zarr", mode='w')

## Building Training Datasets with `ard_gedi.zarr`

In [ ]:
# this is also coppied to the team data store
combined = xr.open_zarr("../glad_ard/ard_gedi.zarr/")

In [ ]:
combined

In [ ]:
# establish the independent and dependent variables
covariates = ["Blue",'Green','Red','NIR','SWIR1','SWIR2','Thermal']
targets = ['rh98']

In [ ]:
# to build a training dataset we subset data to years where we know we
# have gedi points
years = [str(t) for t in range(2019, 2024)]
X_subset = np.array(combined[covariates].isel(time=[-5,-4,-3,-2,-1]).to_array())
Y_subset = np.array(combined[targets].isel(time=[-5,-4,-3,-2,-1]).to_array())

In [ ]:
# since we have some compute to work with and we use a pixel-based modeling
# approach, let's flatten everything to get shape (n,p)
y = Y_subset.reshape(len(targets), -1).T
x = X_subset.reshape(len(covariates), -1).T

# find examples where we have them as GEDI is sparse i.e. wherever data is
# not nan
idx = ~np.isnan(y).any(axis=1)
x = x[idx]
y = y[idx]

# clean both by dropping any nan in x (not y as we just did that above)
valid_x = (~np.isnan(x).any(axis=1))
x = x[valid_x]
y = y[valid_x]

In [ ]:
# how many cleaned examples?
x.shape, y.shape

Training datasets are ready! We have:
- 1072397 examples
- 7 independent variables from GLAD's ARD datasets (landsat bands)
- 1 dependent variable.

## Training
We use the easiest method that is commonly used in the industry--boosted regression. The package we use here is xgboost, which also has nice support for gpu and distributed training on dask, etc. We do zero model development, feature engineering, feature selection, etc.

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

In [ ]:
# we should be splitting spatially! We are going to get optimistic results!
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)  # 80% training and 20% testing

# nuance of xgboost are these DMatrices
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# lots to investigate here!
params = {
    'objective': 'reg:absoluteerror'
}

# more to investigate here
num_round = 100  # the number of training iterations
bst = xgb.train(params, dtrain, num_round)

# run predictions
preds = bst.predict(dtest)

# one simple evaluation metric as a sanity check
print(mean_absolute_error(preds, y_test))

MAE of 3.21 not too bad, but beware of our splitting strategy! We should be estimating this through spatial cross validation for a more rigorous estimation of test error.

Let's at least make a 1:1 plot and compare histograms between predictions and measured for y.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(y_test, bins=80, density=True, alpha=.4, label="measured")
plt.hist(preds, bins=80, color='r', density=True, alpha=.4, label='preds')
plt.xlim(0,40)
plt.legend()
plt.show()

Things to note:
- standard saturation issue for tall trees/things i.e. our predictions don't exhibit the same right skeweness that the measured values do.
- minimum predictions are around 3m as expected with GEDI rh98
- we at least match the bimodal nature of the distribution
- there must be a few outliers since I needed to set the xlim to something reasonable, which means we could possible consider dropping anomalies in our measured rh98 values.
- we are overpredicting rh98 at 15; we can't match the long right-skewed tail; and tend to miss lower predictions as they are shifted to higher values.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KernelDensity
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

max_v = 35

prediction = preds.flatten()
measured = y_test.flatten()
r2 = r2_score(prediction, measured)
rmse = root_mean_squared_error(prediction, measured)
bias = np.mean(measured - prediction)
mae = mean_absolute_error(prediction, measured)

n=10000
idx = np.arange(0, len(prediction))
np.random.shuffle(idx)
prediction = prediction[idx[:n]]
measured = measured[idx[:n]]

# Evaluate KDE at each point
kde = KernelDensity(bandwidth=2)
kde.fit(np.vstack([prediction, measured]).T)
xy = np.vstack([prediction, measured]).T
density = np.exp(kde.score_samples(xy))

# Create scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(prediction, measured, c=density, cmap='viridis', alpha=.8, s=1)
x = np.linspace(min(prediction), max_v, 100)
plt.plot(x, x, color='red', linestyle='--')
plt.xlim(0,max_v)
plt.ylim(0,max_v)

plt.text(
    plt.xlim()[-1]*.05, 
    plt.ylim()[1] * .9, 
    f'R2: {r2:.2f} \nRMSE: {rmse:.2f} \nbias: {bias:.2f} \nMAE: {mae:.2f}',
    verticalalignment='top',
    horizontalalignment='left',
    fontsize=12
)
plt.title("Prediction vs. Measured (rh98)")
plt.xlabel("predicted rh98")
plt.ylabel("measured rh98")
plt.show()


## Estimate `rh98` over all GLAD-ARD 1997-2023
We run the model over all available data now that things are trained. 

In [ ]:
covariates = combined[covariates].to_array().chunk({'x': 256, 'y': 256, 'variable': -1})

In [ ]:
def run_predictions(block):
    """Define a function to apply over blocks"""
    block = np.array(block)
    xx = block.reshape(len(covariates), -1).T
    xx = xgb.DMatrix(xx)
    preds = bst.predict(xx)
    ex = preds.reshape(1, -1).T.reshape(1, *block.shape[1:])
    return ex

In [ ]:
# apply model to each block to make estimates
covariates = covariates.compute()
predictions = covariates.map_blocks(run_predictions)
preds = xr.DataArray(predictions[0], coords=combined[targets].coords)
preds = preds.to_dataset(name='rh98-predictions')
preds.chunk({"time":1, "y":1024,"x": 1024}).to_zarr("../data/predictions.zarr", mode='w')

You can also find `predictions.zarr` up on the team data-store.

### Visualize Predictions over Time

In [ ]:
from geogif import gif

In [ ]:
preds = xr.open_zarr("../data/predictions.zarr")

In [ ]:
gif(preds['rh98-predictions'], to="../data/predictions.gif", fps=2)

In [ ]:
# low res gif for the notebook (slice every 4th pixel in x and y)
gif(preds['rh98-predictions'][:,::4,::4].compute(), fps=2)